# Create MCP Toolbox Configuration
This notebook creates the necessary tools and resources such that this workflow can be replicated in other clients via MCP.

Specifically, this will create the YAML configuration for an [MCP Toolbox server](https://neo4j.com/blog/developer/ai-agents-gen-ai-toolbox/).

Think of these as expert tools - pre-defined query templates with descriptions that can take in optional query parameters. We can later combine this with more generic MCP servers such as [mcp-neo4j-cypher](https://github.com/neo4j-contrib/mcp-neo4j/tree/main/servers/mcp-neo4j-cypher) that allow agents to write and execute their own Cypher queries based on the graph schema and user input.

In [1]:
import getpass
import os
from dotenv import load_dotenv

#get env setup
load_dotenv('nb.env', override=True)

if not os.environ.get('NEO4J_URI'):
    os.environ['NEO4J_URI'] = getpass.getpass('NEO4J_URI:\n')
if not os.environ.get('NEO4J_USERNAME'):
    os.environ['NEO4J_USERNAME'] = getpass.getpass('NEO4J_USERNAME:\n')
if not os.environ.get('NEO4J_PASSWORD'):
    os.environ['NEO4J_PASSWORD'] = getpass.getpass('NEO4J_PASSWORD:\n')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

## Create Mata Context Node
We are actually going to save our former system prompt in the form of a "MetaContext" node.  This allows us to pull and manage source of truth inside our database

In [3]:
from person import Domain, WorkType, SkillName

meta_context = f"""
This knowledge graph, and corresponding tools, provide all the information you need to act as a human resources assistant who helps with skills analysis, talent search, and team formation at Cyberdyne Systems.

Corresponding tools retrieve data from internal knowledge on Cyberdyne System employees based on their resume and profiles.

Try to prioritize expert tools (those other than `read_neo4j_cypher`) as appropriate since they have expert approved logic for access data. Though you may need to directly access data afterwards to pull more details.


When you need more flexible logic for aggregations, follow-up or anything else, you can access the knowledge (in a graph database) directly. ALWAYS get the schema first with `get_schema` and keep it in memory. Only use node labels, relationship types, and property names, and patterns in that schema to generate valid Cypher queries using the `read_neo4j_cypher` tool with proper parameter syntax ($parameter). If you get errors or empty results check the schema and try again at least up to 3 times.

For domain knowledge, use these standard values:
- Domains: {[i.value for i in Domain]}
- Work Types: {[i.value for i in WorkType]}
- Skills: {[i.value for i in SkillName]}

Also never return embedding properties in Cypher queries. This will result in delays and errors.

When responding to the user:
- if your response includes people, include there names and IDs. Never just there Ids.
- You must explain your retrieval logic and where the data came from. You must say exactly how relevance, similarity, etc. was inferred during search

Use information from previous queries when possible instead of asking the user again.
"""

In [7]:
from neo4j import GraphDatabase

#instantiate driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

#test neo4j connection
driver.execute_query("""
MERGE(n:__MetaContext__ {version:1, useCase:'talentAssistant'})
SET n.context = $context
RETURN n
""", context=meta_context)

EagerResult(records=[<Record n=<Node element_id='4:d1805f9b-885c-4575-8474-fe3c00609e7a:357' labels=frozenset({'__MetaContext__'}) properties={'useCase': 'talentAssistant', 'context': "\nThis knowledge graph, and corresponding tools, provide all the information you need to act as a human resources assistant who helps with skills analysis, talent search, and team formation at Cyberdyne Systems.\n\nCorresponding tools retrieve data from internal knowledge on Cyberdyne System employees based on their resume and profiles.\n\nTry to prioritize expert tools (those other than `read_neo4j_cypher`) as appropriate since they have expert approved logic for access data. Though you may need to directly access data afterwards to pull more details.\n\n\nWhen you need more flexible logic for aggregations, follow-up or anything else, you can access the knowledge (in a graph database) directly. ALWAYS get the schema first with `get_schema` and keep it in memory. Only use node labels, relationship types,

## Create MCP ToolBox Definitions
This will create the YAML configuration for an [MCP Toolbox server](https://neo4j.com/blog/developer/ai-agents-gen-ai-toolbox/) so we can expose the tools from previous modules to any MCP client.

In [14]:
# same tools from notebooks 2 & 3
tool_yaml = f"""
sources:
    employee-graph:
        kind: "neo4j"
        uri: "{NEO4J_URI}"
        user: "{NEO4J_USERNAME}"
        password: "{NEO4J_PASSWORD}"
tools:

  get_context:
    kind: neo4j-cypher
    source: employee-graph
    statement: |
        MATCH(n:__MetaContext__ {{version:1, useCase:'talentAssistant'}})
        RETURN n.context AS context
    description: |
        Gets the context for how to use & access employee data. Always run this first and store in your memory.

  find_similar_people:
    kind: neo4j-cypher
    source: employee-graph
    statement: |
        MATCH p=(p1:Person {{id:$personId}})--()
                 ((:!Person)--() ){{0,3}}
                 (p2:Person)
        RETURN count(*) AS score, p2.id AS person_id
        ORDER BY score DESC LIMIT 5 //fixing limit at 5 for now
    description: |
        This function will return potential similar people to the provided person based on common skill and types and domains of accomplishments.  You can use this as a starting point to find similarities scores. But should use follow up tools and queries to collect more info. Returns a list of person ids for similar candidates order by score which is the count of common skill and types and domains of accomplishments
    parameters:
      - name: personId
        type: string
        description: the id of the person to search for similarities for

  find_similarities_between_people:
    kind: neo4j-cypher
    source: employee-graph
    statement: |
        MATCH p=(p1:Person {{id:$person1_id}})--()
                 ((:!Person)--() ){{0,3}}
                 (p2:Person{{id:$person2_id}})
        WITH p, nodes(p) as path_nodes, relationships(p) as path_rels, p1, p2
        RETURN
          "(" + labels(path_nodes[0])[0] + " {{name: \\"" + path_nodes[0].name + "\\" id: \\"" + path_nodes[0].id + "\\"}})" +
          reduce(chain = "", i IN range(0, size(path_rels)-1) |
            chain +
            "-[" + type(path_rels[i]) + "]-" +
            "(" + labels(path_nodes[i+1])[0] + " {{name: \\"" + path_nodes[i+1].name +
            CASE WHEN "Person" IN labels(path_nodes[i+1])
                 THEN "\\" id: \\"" + path_nodes[i+1].id +"\\""
                 ELSE "\\"" END + "}})"
          ) as paths ORDER BY p1.id, p2.id
    description: |
        This function will return potential similarities between people in the form of skill and accomplishment paths.  You can use this as a starting point to find similarities and query the graph further using the various name fields and person ids. Returns a list of paths between the two people, each path is a compact ascii string representation.  It should reflect the patterns in the graph schema.
    parameters:
      - name:  person1_id
        type: string
        description: the id of the first person to compare
      - name:  person2_id
        type: string
        description: the id of the second person to compare

  get_person_resume:
    kind: neo4j-cypher
    source: employee-graph
    statement: |
        MATCH (n:Person {{id: $personId}})
        RETURN n.text as resume, n.name AS name
    description: Gets the full resume of a person and their name.
    parameters:
      - name: personId
        type: string
        description: the id of the person

  get_person_name:
    kind: neo4j-cypher
    source: employee-graph
    statement: |
        MATCH (n:Person {{id: $personId}})
        RETURN n.name
    description: Gets a person name given their id
    parameters:
      - name: personId
        type: string
        description: the id of the person

  get_person_ids_from_name:
    kind: neo4j-cypher
    source: employee-graph
    statement: |
        MATCH (n:Person {{name: $personName}})
        RETURN n.id
    description: |
        Gets all the unique person ids who have the provided name. Note that names aren't guaranteed to be unique so you may get more than one person.
    parameters:
      - name: personName
        type: string
        description: the name to look up person ids with

  find_collaborators_in_domain:
    kind: neo4j-cypher
    source: employee-graph
    statement: |
        // graph pattern: get collaborates in set of domains
        MATCH (p:Person)-[r]->(t:Thing)<-[]-(:Person)
        MATCH (t)-[:IN]->(d)
        MATCH (t)-[:OF]->(w)
        WHERE d.name IN $domains
        //format for response
        WITH t, d, w, p,
        collect({{contribution: type(r) + " IT", roleDetails: r.role, duration:r.duration}}) AS contributions
        WITH t.name AS projectName, t.internal_project AS is_internal, d.name AS domain, w.name AS workType, collect({{name: p.name, id: p.id, contributions: contributions}}) AS people
        //return
        RETURN projectName, domain, workType, people
    description: |
        Finds projects within a given set of domains where multiple people collaborated
        along with project details and individual contributions.
        It returns detailed information about each project, including the project name,
        domain, work type, and the list of collaborators (their names, IDs, and their contributions).
    parameters:
      - name: domains
        type: array
        description: |
            A list of domains (from the `Domain` enum) to filter projects where collaborations occurred.
        items:
          name: name
          type: string
          description: Domain name. Valid values are {', '.join([i.value for i in Domain])}.
"""
with open('tools.yaml', 'w') as file:
    file.write(tool_yaml)
print("tools.yaml file created successfully.")

tools.yaml file created successfully.


## Deploy MCP Toolbox
You can do this locally or, if you have a GCP account, deploy remotely via Cloud Run.

For local deployment see `deploy-toolbox-local.sh`. For remote GCP see `deploy-toolbox-gcp.sh`.

Optional: Once the Toolbox MCP server is running you can test with [MCP Inspector](https://modelcontextprotocol.io/legacy/tools/inspector).